In [1]:
import os
import numpy
from datetime import date
import subprocess
import sys  
import operator
import random
import re
import warnings
import math
from itertools import combinations
from IPython.display import display, clear_output, HTML

favCutoff = "40"

import numpy as np
from scipy import stats
from sklearn.decomposition import PCA
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import seaborn as sns

import ete3
from ete3 import NCBITaxa
ncbi = NCBITaxa()

from Bio.Blast import NCBIWWW, NCBIXML
from Bio import SeqIO, Entrez
Entrez.email = "parsonsc@mit.edu"
Entrez.api_key = "7a3fcee8ce251bc6a20619259badbd6f0e09"

#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

#!{sys.executable} -m pip install --user biopython
#!{sys.executable} -m pip install --user ete3

In [ ]:
def makeAlignmentOld(clusterName, seqIDs):
    completed = subprocess.run(["mkdir", "fullTrees/"])
    completed = subprocess.run(["mkdir", "fullTrees/cluster" + favCutoff])
    if len(clusterName.split("_")) > 2:
        clusterName = "_".join(clusterName.split("_")[:2])
    alignment_name = clusterName + ".fasta"
    if filename in os.listdir("fullTrees/cluster" + favCutoff):
        print("File already found! (%s)"%(filename))
        raise Exception
    
    count = len(seqIDs)
    out = open("fullTrees/cluster" + favCutoff + "/" + filename, 'w')
    batch_size = 500
    for start in range(0, count, batch_size):
        end = min(count, start+batch_size)
        ids = ','.join(seqIDs[start:end])
        try:
            handle = Entrez.efetch(db="protein", rettype="fasta", retmode="text", id=ids)
            out.write(handle.read())
        except:
            continue
        handle.close()
    out.close()
    
    print(count, "sequences downloaded.")
    
    name = "mafft_" + date.today().strftime("%m_%d_%Y") + "_" + clusterName
    commands = ["module add engaging/mafft",
                "mafft --localpair --maxiterate 1000 fullTrees/cluster" + favCutoff + 
                "/" + filename + " > alignments/cluster" + favCutoff + "/" + clusterName + ".afa"]
    mafftSh = makeShell(name, commands, mem = '10')
    print(mafftSh)

    completed = subprocess.run(["sbatch",  "--wait",  mafftSh])
    
def makeFullTree(clusterName):
    alignmentName = "fullTrees/cluster" + favCutoff + "/" + clusterName + ".afa"
    name = "iqtree_" + date.today().strftime("%m_%d_%Y") + "_" + clusterName

In [ ]:
def fastaToDictionary(alignmentName):
    alignFile = open(alignmentName)
    alines = alignFile.readlines()
    alignFile.close

    seqDict = {}
    cur = ""

    for line in alines:
        l = line.strip()
        if l == '':
            continue
        if l[0] == '>':
            cur = l[1:]
            seqDict[cur] = ""
        else:
            if cur not in seqDict:
                print(alignmentName, l)
            seqDict[cur] += l

    return seqDict

def makeShell(name, commands, nodes='1', cores='1', mem='10', days='7', max_queue='20', out=None):
    if out == None:
        out = name
    if max_queue:
        subprocess.call(["while [ $( squeue -u parsonsc | wc -l) -gt " + max_queue + " ]; do\n    sleep 10; done;"], shell=True)
    file = open(name + '.sh', 'w')
    file.write( "#!/bin/bash\n\n"
                "#SBATCH -p sched_mit_g4nier\n"                                                                          
                "#SBATCH -t " + days + "-00:00:00\n"
                "#SBATCH -N " + nodes + "\n"
                "#SBATCH -n " + cores + "\n"
                "#SBATCH --mem=" + mem + "G\n"
                "#SBATCH -J " + name + "\n"
                "#SBATCH -o " + out + ".out\n" +
                '\n'.join(commands))
    file.close()
    return name + '.sh'

def removeRedundancy(fileName):
    lines = [x.strip() for x in open(fileName).readlines()]
    
    seqDict = {}
    
    cur = ''
    running = ''
    for line in lines:
        if line[0] == '>':
            if cur != '':
                if running not in seqDict.values():
                    seqDict[cur] = running
            cur = line[1:]
            running = ''
        else:
            running += line
    if running not in seqDict.values():
        seqDict[cur] = running
    
    out = open(filename, 'w')
    for seq in seqDict:
        out.write(seq + '\n' + seqDict[seq] + '\n')
    out.close()
        
def getSis(node):
    if node.is_root():
        return False
    else:
        return [x for x in node.up.children if x != node][0]
    
def getClade(node):
    leaves = node.get_leaves()
    oneEx = leaves[0].name[0]
    if all(x.name[0] == oneEx for x in leaves):
        return oneEx
    else:
        return None
        
# MAD file generated with -tf flag
def addMADtoTree(treeFile, rootedTreeFile):
    f = open(treeFile)
    treeLine = f.readlines()[0]
    f.close()
    rf = open(rootedTreeFile)
    rTreeLine = rf.readlines()[2][19:].replace(",ADS=", "comADS=").replace(",STT=", "comSTT=").split("[&ADS=\"AI")[0] + ";"
    rf.close()

    tree = ete3.Tree(treeLine, format=1)
    rTree = ete3.Tree(rTreeLine, format=1)
    if len(rTree.children[0]) == 1:
        rTree = rTree.children[1]
    else:
        rTree = rTree.children[0]
    rTreeOg = rTree.children[0]
    if len(rTreeOg) == 1:
        rTreeOg = rTree.children[1]
    for leaf in rTree.get_leaves():
        leaf.add_feature("label", "[" + leaf.name.split("[")[1])
        leaf.name = leaf.name.split("[")[0]
    ogNames = [x.name for x in rTreeOg.get_leaves()]
    for leaf in tree.get_leaves():
        if leaf.name not in ogNames:
            tree.set_outgroup(leaf)
            break
    treeOg = tree.get_common_ancestor(ogNames)
    
    tree.set_outgroup(treeOg)
    treeNodes = list(tree.traverse())
    rTreeNodes = list(rTree.traverse())
    
    for node in tree.traverse():
        leafNames = [x.name for x in node.get_leaves()]
        if len(node) == 1:
            rNode = rTree&leafNames[0]
            adVal = rNode.label.split("com")[0].split("=")[-1]
            adsVal = rNode.label.split("com")[1].split("=")[-1]
        else:
            rNode = rTree.get_common_ancestor(leafNames)
            adVal = rNode.name.split("com")[0].split("=")[-1]
            adsVal = rNode.name.split("com")[1].split("=")[-1]
        if len(rNode) != len(node):
            print("Something wrong lol", len(rNode), len(node))
        node.add_features(AD=adVal, ADS=adsVal)
    return tree


In [ ]:
def makeAlignment(clusterName, seqIDs, max_queue="20"):
    completed = subprocess.run(["mkdir", "fullTrees/"])
    completed = subprocess.run(["mkdir", "fullTrees/cluster" + favCutoff])
    completed = subprocess.run(["mkdir", "fullTrees/cluster" + favCutoff + "/alignments"])

    filename = clusterName + ".fasta"
    if filename in os.listdir("fullTrees/cluster" + favCutoff):
        print("Fasta found! (%s)"%(filename))
    else:
        count = len(seqIDs)
        out = open("fullTrees/cluster" + favCutoff + "/" + filename, 'w')
        batch_size = 500
        for start in range(0, count, batch_size):
            end = min(count, start+batch_size)
            ids = ','.join(seqIDs[start:end])
            try:
                handle = Entrez.efetch(db="protein", rettype="fasta", retmode="text", id=ids)
                out.write(handle.read())
            except:
                continue
            handle.close()
        out.close()
    
    #print(count, "sequences downloaded.")
    
    if filename[:-5] + "afa" in os.listdir("fullTrees/cluster" + favCutoff + "/alignments/"):
        print("Alignment already found! (%s)"%(filename[:-5] + "afa"))
    else:
        name = "mafft_" + date.today().strftime("%m_%d_%Y") + "_" + clusterName
        commands = ["module add engaging/mafft",
                    "mafft --retree 2 --maxiterate 1000 fullTrees/cluster" + favCutoff + 
                    "/" + filename + " > fullTrees/cluster" + favCutoff + "/alignments/" + filename[:-5] + "afa"]
        mafftSh = makeShell(name, commands, mem = '10', max_queue=max_queue)
        #print(mafftSh)
        completed = subprocess.run(["sbatch", mafftSh])
        #completed = subprocess.run(["sbatch",  "--wait",  mafftSh])
    
def makeFullTree(clusterName):
    alignmentName = "fullTrees/cluster" + favCutoff + "/" + clusterName + ".afa"
    name = "iqtree_" + date.today().strftime("%m_%d_%Y") + "_" + clusterName


def generate_small_alignments(cluster, blast_path, n=10, batch_size=300, 
                              full_tree=False, skip_duplicates=False, return_tax=False):
    seq_ids = {}
    seq_tax = {}
    with open(blast_path) as f:
        prev_tid = None
        for line in f:
            parts = [x.strip() for x in line.split()]
            if cluster in parts[1].split(","):
                if skip_duplicates and (parts[2] == prev_tid):
                    continue
                else:
                    prev_tid = parts[2]
                    seq_ids[parts[0]] = parts[3]
                    seq_tax[parts[0]] = parts[2]
    #met = [x for x in seq_ids if seq_ids[x]=="m"]
    #bac = [x for x in seq_ids if seq_ids[x]=="b"]
    #nei = [x for x in seq_ids if seq_ids[x]=="n"]
    #mbn = [len(met),len(bac),len(nei)]
    #print("%s -- metazoa:%i,bacteria:%i,other:%i" %(cluster,len(met),len(bac),len(nei)))
    for i in range(n):
        trial_name = "%s_%i"%(cluster, i+1)
        sample = random.choices(met, k=int(batch_size*(5/12))) + random.choices(bac, k=int(batch_size*(5/12))) + random.choices(nei, k=int(batch_size*(1/6)))
        makeAlignment(trial_name, sample)
    if full_tree:
        makeAlignment("%s_full"%(cluster), met+bac+nei)
    if return_tax:
        return seq_ids, seq_tax, mbn
    else:
        return seq_ids, seq_tax 

In [ ]:
def update_default_names(alignment_path, taxdict, groupdict):
    old = fastaToDictionary(alignment_path)
    #print([x for x in old])
    oldsplit = {x.split()[0]:old[x] for x in old}
    oldsplit = {x:oldsplit[x] for x in oldsplit}
    new = {"|".join([groupdict[x], taxdict[x], x]):oldsplit[x] for x in oldsplit if x in groupdict}
    #print(len(old), len(oldsplit), len(new))
    with open(alignment_path.replace(".afa", "_rn.afa"), 'w') as out:
        for seq in new:
            out.write(">" + seq + "\n" + new[seq] + "\n")

In [ ]:
# Infer metazoan-bacterial distance

def calculateMBDistance(tree):
    tree = tree.copy()
    mAndB = []
    for leaf in tree.get_leaves():
        #iqtree turns | into _
        if "|" not in leaf.name:
            parts = leaf.name.split("_")
            leaf.name = "|".join(parts[0:2] + ["_".join(parts[2:])])
        if leaf.name[0] in ['m', 'b']:
            mAndB.append(leaf)
    tree.prune(mAndB, preserve_branch_length=True)

    leaves = tree.get_leaves()
    animalRoot = next(x for x in leaves if x.name[0] == 'm')
    tree.set_outgroup(animalRoot)

    # Label the easy ones (e.g. nodes with all bacterial or metazoan children)
    for node in tree.traverse():
        if not node.is_leaf():
            clade = getClade(node)
            node.add_feature("nodeName", node.name.split('/')[0])
            if node.is_root():
                node.name = 'r'
            elif clade:
                node.name = clade

    # Loops until all internal nodes are labelled
    while not all(x.name in ['b', 'm', 'u', 'r'] or x.is_leaf() for x in tree.traverse()):
        # Grabs all the already-labelled internal nodes
        toLookAt = [x for x in tree.traverse() if 
                    (x.name in ['b', 'm', 'u']) or x.is_leaf()]
        mut = []
        for node in toLookAt:
            par = node.up
            # If the parent's already labelled, continue
            if par.name in ['b', 'm', 'u', 'r']:
                continue

            # If the sister doesn't have a name, continue
            sis = getSis(node)
            if sis.is_leaf():
                sisName = sis.name[0]
            elif sis.name in ['b', 'm', 'u']:
                sisName = sis.name
            else:
                continue

            if node.is_leaf():
                nodeName = node.name[0]
            else:
                nodeName = node.name

            # If the node and its sister have the same label, give the parent that label
            if nodeName == sisName:
                par.name = nodeName
            # Use the parent's sister to infer parent's label, if node and sister
            # have conflicting labels
            else:
                parSis = getSis(par)
                if parSis.is_leaf():
                    parSisName = parSis.name[0]
                else:
                    parSisName = parSis.name
                # Hopefully parent's sister is labelled
                if parSisName in ['b', 'm', 'u']:
                    # If parent's sister matches one of the parent's children,
                    # parent is assigned that label
                    if parSisName in [nodeName, sisName]:
                        par.name = parSisName
                    # Propagate unknown (e.g ((M,B),U) or ((M,U),B))
                    else:
                        par.name = 'u'
                # If parent's sister is unlabelled, either wait until it is,
                # or recognize that neither can inform each other and label
                # as unknown (e.g. ((M,B),(M,B)))
                else:
                    if [parSis,par] in mut:
                        par.name = 'u'
                        parSis.name = 'u'
                    else:
                        mut.append([par, parSis])

    # Make the OG a bacterial subtree
    for node in [x for x in tree.traverse() if x.name == "b"]:
        if node.up.name == "m":
            tree.set_outgroup(node)
            break

    tree.set_outgroup(animalRoot)
    dists = []
    for node in [x for x in tree.traverse() if not (x.is_root() or x.is_leaf())]:
        par = node.up
        if node.name == 'u' or par.name == 'u' or par.name == 'r':
            continue
        kidNames = [x.name.split('|')[1] for x in node.get_leaves()]
        if len(list(set(kidNames))) == 1:
            #print(node)
            continue
        if node.name != par.name:
            dists.append(node.dist)
            #print(node.dist)
    if dists == []:
        #print("Too uncertain")
        return []
    else:
        return dists
    #else:
    #    tree.write(format=1,outfile="uhOh2.tree")

In [ ]:
# Infer taxonomic coverage
# A high ratio of unique genera to unique phyla implies vertical inheritance
def inferTaxonomicCoverage(blastHitsFile, modifier=None, tid_in_lineage=None):
    f = open(blastHitsFile)
    flines = [line.strip() for line in f]
    f.close()

    all_taxa = {"phylum":set(), "class":set(), "order":set(), "family":set(), "genus":set()}
    cluster_taxa = {}

    seen = {}
    count = 0
    for line in flines:
        if count % 1000 == 0:
            clear_output(wait=True)
            print(count, round(count/len(flines)*100,2), len(all_taxa["phylum"]), len(all_taxa["genus"]))
        count += 1
        seq, clust, tids, dom = line.split()
        if (not (modifier is None)) and dom != modifier:
            continue
        tid = tids.split(',')[0]
        if tid in seen:
            lineage, ranks = seen[tid]
            new = False
        else:
            try:
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    lineage = ncbi.get_lineage(tid)
            except:
                print("Invalid taxid:", tid)
                seen[tid] = ([], {})
                continue
            if (not (tid_in_lineage is None)) and tid_in_lineage not in lineage: 
                lineage, ranks = [], {}
            else:
                ranks = ncbi.get_rank(lineage)
                lineage = [x for x in lineage if ranks[x] in all_taxa]
                ranks = {x:ranks[x] for x in ranks if x in lineage}
            new = True
            seen[tid] = (lineage, ranks)
        for lin_tid in lineage:
            rank = ranks[lin_tid]
            clusts = clust.split(',')
            for c in clusts:
                if c not in cluster_taxa:
                    cluster_taxa[c] = {"phylum":set(), "class":set(), "order":set(), "family":set(), "genus":set()}
                #if lin_tid not in cluster_taxa[c][rank]:
                cluster_taxa[c][rank].add(lin_tid)
            if new: #and lin_tid not in all_taxa[rank]:
                all_taxa[rank].add(lin_tid)
                
    all_taxa = {x:list(all_taxa[x]) for x in all_taxa}
    for c in cluster_taxa:
        cluster_taxa[c] = {x:list(cluster_taxa[c][x]) for x in cluster_taxa[c]}
    return all_taxa, cluster_taxa

In [1]:
def combine_fastas(fastas, out_path):
    out = open(out_path, 'w')
    seen = []
    stop = False
    for fasta_path in fastas:
        with open(fasta_path) as f:
            for line in f.readlines():
                l = line.strip()
                if l and l[0]==">":
                    if l in seen:
                        stop = True
                        continue
                    stop = False
                    seen.append(l)
                if not stop:
                    out.write(l+"\n")
    out.close()
        
        
        
        

In [ ]:
# Write all trees to single figtree NEXUS file for visualization
def merge_and_write_trees(cluster, trees,out_path, sort_by=None, linDict={}, rankDict={}, nameDict={}, reuse=False):
    #linDict = {}
    #rankDict = {}
    #nameDict = {}
    favCutoff = "40"
    taxLines = []
    treeLines = []
    #maxDists = {}
    #for file in dists:
    #    maxDists[file] = max(dists[file])
    if sort_by:
        sort_by_tuples = [(x,sort_by[x]) for x in sort_by]
        sorted_trees = sorted(sort_by_tuples.items(), key=operator.itemgetter(1))
        sorted_trees = [x[0] for x in sorted_trees]
    else:
        sorted_trees = trees
    #out = open("trees/cluster" + favCutoff + "/geneNames.txt", 'w')
    #for file, dist in sorted_trees:
    #    if file[0] == "h":
    #        continue
    #    out.write(file + "," + clustAnnotations[file] + "," + " ".join([str(x) for x in dists[file]]) + "\n")
    #out.close()

    for file in sorted_trees:
        print(file)
        #if file[0] == "h":
        #    continue
        tree = trees[file]
        taxTree = tree.copy()
        for node in taxTree.traverse():
            if node.is_leaf():
                parts = node.name.split('|')
                taxid = int(parts[1].split(',')[0])
                gen_spe = nameDict.get(taxid)
                if gen_spe is None:
                    try:
                        gen_spe = ncbi.get_taxid_translator([taxid])[taxid]
                    except:
                        taxLine = '\t%s [&gen_spe="%s"]' %(node.name, "idk")
                        taxLines.append(taxLine)
                        continue
                taxLine = '\t%s [&gen_spe="%s"' %(node.name, gen_spe)

                lin = linDict.get(taxid)
                if not lin:
                    lin = ncbi.get_lineage(taxid)
                    linDict[taxid] = lin
                notIn = [x for x in lin if x not in nameDict]
                names = ncbi.get_taxid_translator(notIn)
                ranks = ncbi.get_rank(notIn)
                for name in names:
                    nameDict[name] = names[name]
                    rankDict[name] = ranks[name]
                lineage = dict([(rankDict[x], nameDict[x]) for x in lin])
                for rank in ['phylum', 'class', 'order', 'family']:
                    if rank in lineage:
                        taxLine += ' tax_%s="%s"' %(rank, lineage[rank])
                if 2 in lin: #bacteria
                    dom = "bac"
                elif 2157 in lin: #archaea
                    dom = "arc"
                elif 2759 in lin: #eukaryota
                    #if 33213 in lin:
                    #    dom = "bil"
                    if 33208 in lin: #metazoa
                        dom = "met"
                    elif 4751 in lin: #fungi
                        dom = "fun"
                    elif 33090 in lin: #viridiplantae
                        dom = "pla"
                    else:
                        dom = "euk" 
                else: #other
                    dom = "nan"
                taxLine += ' group="%s"' %(dom)
                taxLine += ' fullTax="%s"' %'_'.join([nameDict[x] for x in lin])
                taxLine += ']'
                taxLines.append(taxLine)
                #print(dom)
            else:
                if node.name:
                    if '/' in node.name:
                        write_format = 1
                        bits = node.name.split('/')
                        aLRT, UFBoot = node.name.split('/')
                        node.name = '[&nodeName=%s,UFBoot=%.2f,aLRT=%.2f]' %(node.name, float(UFBoot), float(aLRT))
                    else:
                        write_format = 0
                        node.support = float(node.name)

        newick_text = taxTree.write(format=write_format)
        #print(newick_text)
        if write_format>0:
            regBit = '_&nodeName_(\d+\.?\d?\/\d+)_UFBoot_(\d+\.\d\d)_aLRT_(\d+\.\d\d)_'
            #regBit = '_&nodeName_(Node\d+)_UFBoot_(\d+\.\d\d)_aLRT_(\d+\.\d\d)_AD_(\d+\.\d+)_'
            newLab = '[&nodeName=\\1,UFBoot=\\2,aLRT=\\3]'
            #newLab = '[&nodeName=\\1,UFBoot=\\2,aLRT=\\3,AD=\\4]'
            newick_text = re.sub(regBit, newLab, newick_text)
        #print(newick_text)
        treeLines.append("\ttree %s = [&R] %s" %(file, newick_text))

    out = open(out_path, 'w')
    out.write("#NEXUS\nbegin taxa;\n\tdimensions ntax=%i;\n\ttaxlabels\n" %len(taxLines))
    out.write('\n'.join(taxLines))
    out.write(';\nend;\n')
    out.write('begin trees;\n%s\nend;' %'\n'.join(treeLines))
    out.write("begin figtree;\n\tset tipLabels.colorAttribute=\"group\";")
    out.write("\n\tset tipLabels.displayAttribute=\"tax_phylum\";\nend;")
    out.close()
    if reuse:
        return linDict, rankDict, nameDict

In [ ]:
def update_all_names(alignment_path):
    alignments = [x for x in os.listdir(alignment_path) if "afa" == x.split(".")[-1] and os.path.getsize(alignment_path+x)>0]
    aligned_clusters = list(set(["_".join(x.split("_")[:2]) for x in alignments]))
    for clust in aligned_clusters:
        sub_aligns = [x for x in alignments if clust in x and "_rn.afa" not in x]
        sub_aligns = [x for x in sub_aligns if x.replace(".afa","_rn.afa") not in alignments]
        if len(sub_aligns) == 0:
            continue
        groupdict, taxdict = generate_small_alignments(clust, blast_path, n=0, batch_size=0)
        for file in sub_aligns:
            print("Renaming", file)
            update_default_names(alignment_path+file, taxdict, groupdict)

In [ ]:
def split_blast_by_domain(blast_path, unique=False, clusters_of_interest=None, write_path=False):
    #cluster_counts = {}
    cluster_groups = {}

    seen = {}
    tax_ids = {2157:"a", 33090:"p", 4751:"f"}
    lines = []
    with open(blast_path) as f:
        for line in f:
            parts = [x.strip() for x in line.split()]
            dom = parts[3]
            tid = parts[2]
            tid = tid.split(",")[0]
            clusters = parts[1].split(",")
            for cluster in clusters:
                if clusters_of_interest:
                    if cluster not in clusters_of_interest:
                        continue
                if cluster not in cluster_groups:
                    #cluster_counts[cluster] = {tax_ids[x]:0 for x in tax_ids}
                    cluster_groups[cluster] = {tax_ids[x]:[] for x in tax_ids}
                if dom not in cluster_groups[cluster]:
                    #cluster_counts[cluster][dom] = 0
                    cluster_groups[cluster][dom] = []
                #cluster_counts[cluster][dom] += 1
                cluster_groups[cluster][dom].append(tid)
                if dom == "n":
                    if tid in seen:
                        lin = seen[tid]
                    else:
                        try:
                            lin = ncbi.get_lineage(tid)
                        except:
                            lin = []
                        seen[tid] = lin
                    for tax_id in tax_ids:
                        if tax_id in lin:
                            dom = tax_ids[tax_id]
                            #cluster_counts[cluster][dom] += 1
                            cluster_groups[cluster][dom].append(tid)
            if write_path:
                parts[3] = dom
                lines.append("\t".join(parts))
    if write_path:
        out = open(write_path, 'w')
        out.write("\n".join(lines))
        out.close()
    cluster_counts = {}
    for cluster in cluster_groups:
        if unique:
            cluster_groups[cluster] = {x:list(set(cluster_groups[cluster][x])) for x in cluster_groups[cluster]}
        cluster_counts[cluster] = {x:len(cluster_groups[cluster][x]) for x in cluster_groups[cluster]}
    return cluster_counts


In [ ]:
def calc_range(values, exclude_outliers=False):
    if exclude_outliers:
        q75, q25 = np.percentile(values, [75 ,25])
        iqr = q75 - q25
        too_high = [x for x in values if x>(q75+(1.5*iqr))]
        too_low = [x for x in values if x<(q25-(1.5*iqr))]
        #print(values, too_high, too_low)
        values = [x for x in values if x not in too_high+too_low]
    return max(values) - min(values)

def calc_iqr(values):
    q75, q25 = np.percentile(values, [75 ,25])
    return q75 - q25

In [ ]:
def list_with_separators_to_count_dict(list_with_separators, separator=","):
    out = {}
    for item in list_with_separators:
        for sub_item in item.split(separator):
            if sub_item in out:
                out[sub_item]+=1
            else:
                out[sub_item]=1
    return out